In [5]:
import pandas as pd
import numpy as np
from glob import glob
import multiprocessing
import pickle
import pandas as pd
import urllib.request
import json
import datetime

days = []
today = datetime.datetime.today()
yesterday = (today - datetime.timedelta(1)).strftime('%Y%m%d') 
day_frame = pd.DataFrame()
dt_index = pd.date_range(start='20211104', end='20211104')
dt_list = dt_index.strftime("%Y%m%d").tolist()

for i in dt_list:
    days.append(i)    

for day in days:
    url = 'https://www.nongnet.or.kr/api/whlslDstrQr.do?sdate=' # sdate = 날짜

    response = urllib.request.urlopen(url+day).read()
    response = json.loads(response)
    data = pd.DataFrame(response['data'])
    if len(data) < 1:
        sample = pd.read_csv('./다운로드/235801_2021 농산물 가격예측 AI 경진대회/public_data/datas/20210824.csv')
        data = pd.DataFrame(columns = [sample.columns[1:]])
        data.loc[0] = 0
        data['SALEDATE'] = day
    data.to_csv(f'./다운로드/235801_2021 농산물 가격예측 AI 경진대회/public_data/datas/{day}.csv')

In [6]:
# 윈도우즈 사용시 함수를 별도의 .py파일로 저장 후 import하여 사용
def preprocessing(tsalet_file):
    unique_pum = [
        '배추', '무', '양파', '건고추','마늘',
        '대파', '얼갈이배추', '양배추', '깻잎',
        '시금치', '미나리', '당근',
        '파프리카', '새송이', '팽이버섯', '토마토',
    ]
    
    unique_kind = [
        '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
    ]
    
    train_dict = {
        'date':[]
    }
    """
    QTY: 물량
    COST: 단가
    """
    
    for sub in unique_pum:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        train_dict[f'{sub}_COSTm'] = []
        train_dict[f'{sub}_COSTs'] = []
        train_dict[f'{sub}_SAN_NM'] = []
        train_dict[f'{sub}_WHSAL_NM'] = []
        train_dict[f'{sub}_CMP_NM'] = []
        train_dict[f'{sub}_KIND_NM'] = []
        train_dict[f'{sub}_QTYs'] = []
        train_dict[f'{sub}_QTYm'] = []
        train_dict[f'{sub}_DANQs'] = []
        train_dict[f'{sub}_DANQm'] = []
        train_dict[f'{sub}_LV_NM'] = []
        
    for sub in unique_kind:
        train_dict[f'{sub}_거래량(kg)'] = []
        train_dict[f'{sub}_가격(원/kg)'] = []
        train_dict[f'{sub}_COSTm'] = []
        train_dict[f'{sub}_COSTs'] = []
        train_dict[f'{sub}_SAN_NM'] = []
        train_dict[f'{sub}_WHSAL_NM'] = []
        train_dict[f'{sub}_CMP_NM'] = []
        train_dict[f'{sub}_KIND_NM'] = []
        train_dict[f'{sub}_QTYs'] = []
        train_dict[f'{sub}_QTYm'] = []
        train_dict[f'{sub}_DANQs'] = []
        train_dict[f'{sub}_DANQm'] = []
        train_dict[f'{sub}_LV_NM'] = []
        
    tsalet_sample = pd.read_csv(tsalet_file)
    days = sorted(tsalet_sample['SALEDATE'].unique())
    for day in days:
        train_dict['date'].append(day)
        for sub in unique_pum:
            # 날짜별, 품목별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['PUM_NM']==sub) & (tsalet_sample['TOT_QTY']>=0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
                train_dict[f'{sub}_SAN_NM'].append(0)
                train_dict[f'{sub}_WHSAL_NM'].append(0)
                train_dict[f'{sub}_CMP_NM'].append(0)
                train_dict[f'{sub}_COSTm'].append(0)
                train_dict[f'{sub}_COSTs'].append(0)
                train_dict[f'{sub}_KIND_NM'].append(0)
                train_dict[f'{sub}_QTYs'].append(0)
                train_dict[f'{sub}_QTYm'].append(0)
                train_dict[f'{sub}_DANQs'].append(0)
                train_dict[f'{sub}_DANQm'].append(0)
                train_dict[f'{sub}_LV_NM'].append(0)
            else:
                sans = c['SAN_NM'].unique()[0]

                whsal = c['WHSAL_NM'].unique()[0]

                cmp = c['CMP_NM'].unique()[0]

                kind = c['KIND_NM'].unique()[0]

                lv = c['LV_NM'].unique()[0]

                qtys = c['QTY'].sum().astype(float)
                qtym = c['QTY'].mean().astype(float)
                dans = c['DANQ'].sum().astype(float)
                danm = c['DANQ'].mean().astype(float)
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                costm = c['COST'].mean().astype(float)
                costs = c['COST'].sum().astype(float)
                mean_price = tot_amt/(tot_qty+1e-20)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                train_dict[f'{sub}_SAN_NM'].append(sans)
                train_dict[f'{sub}_WHSAL_NM'].append(whsal)
                train_dict[f'{sub}_CMP_NM'].append(cmp)
                train_dict[f'{sub}_COSTm'].append(costm)
                train_dict[f'{sub}_COSTs'].append(costs)
                train_dict[f'{sub}_KIND_NM'].append(kind)
                train_dict[f'{sub}_LV_NM'].append(lv)
                train_dict[f'{sub}_QTYs'].append(qtys)
                train_dict[f'{sub}_QTYm'].append(qtym)
                train_dict[f'{sub}_DANQs'].append(dans)
                train_dict[f'{sub}_DANQm'].append(danm)

        
        for sub in unique_kind:
            # 날짜별, 품종별, 거래량이 0 이상인 행만 선택
            c = tsalet_sample[(tsalet_sample['SALEDATE']==day) & (tsalet_sample['KIND_NM']==sub) & (tsalet_sample['TOT_QTY']>0)]
            if c.shape[0] == 0:
                train_dict[f'{sub}_거래량(kg)'].append(0)
                train_dict[f'{sub}_가격(원/kg)'].append(0)
                train_dict[f'{sub}_SAN_NM'].append(0)
                train_dict[f'{sub}_WHSAL_NM'].append(0)
                train_dict[f'{sub}_CMP_NM'].append(0)
                train_dict[f'{sub}_COSTm'].append(0)
                train_dict[f'{sub}_COSTs'].append(0)
                train_dict[f'{sub}_KIND_NM'].append(0)
                train_dict[f'{sub}_QTYs'].append(0)
                train_dict[f'{sub}_QTYm'].append(0)
                train_dict[f'{sub}_DANQs'].append(0)
                train_dict[f'{sub}_DANQm'].append(0)
                train_dict[f'{sub}_LV_NM'].append(0)
            else:
                sans = c['SAN_NM'].unique()[0]
                whsal = c['WHSAL_NM'].unique()[0]
                cmp = c['CMP_NM'].unique()[0]
                kind = c['KIND_NM'].unique()[0]
                lv = c['LV_NM'].unique()[0]
                qtys = c['QTY'].sum().astype(float)
                qtym = c['QTY'].mean().astype(float)
                dans = c['DANQ'].sum().astype(float)
                danm = c['DANQ'].mean().astype(float)
                tot_amt = c['TOT_AMT'].sum().astype(float)
                tot_qty = c['TOT_QTY'].sum().astype(float)
                costm = c['COST'].mean().astype(float)
                costs = c['COST'].sum().astype(float)
                mean_price = round(tot_amt/(tot_qty+1e-20))
                tot_qty = round(tot_qty, 1)
                train_dict[f'{sub}_거래량(kg)'].append(tot_qty)
                train_dict[f'{sub}_가격(원/kg)'].append(mean_price)
                train_dict[f'{sub}_SAN_NM'].append(sans)
                train_dict[f'{sub}_WHSAL_NM'].append(whsal)
                train_dict[f'{sub}_CMP_NM'].append(cmp)
                train_dict[f'{sub}_COSTm'].append(costm)
                train_dict[f'{sub}_COSTs'].append(costs)
                train_dict[f'{sub}_KIND_NM'].append(kind)
                train_dict[f'{sub}_LV_NM'].append(lv)
                train_dict[f'{sub}_QTYs'].append(qtys)
                train_dict[f'{sub}_QTYm'].append(qtym)
                train_dict[f'{sub}_DANQs'].append(dans)
                train_dict[f'{sub}_DANQm'].append(danm)
                
    with open(f'./다운로드/235801_2021 농산물 가격예측 AI 경진대회/public_data/dict/{tsalet_file[-12:-4]}.pkl', 'wb') as f:
        pickle.dump(train_dict, f)

In [7]:
import re

def processing(x):
    x = str(x)
    x = x[0:4] + x[5:7] + x[8:10]
    return x

def processing_date(x):
    x = str(x)
    x = x[0:4] + "-" + x[4:6] + "-" + x[6:8]
    return x

tsalet_files = sorted(glob('./다운로드/235801_2021 농산물 가격예측 AI 경진대회/public_data/datas/*'))

pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
pool.map(preprocessing, tsalet_files)
pool.close()
pool.join()

dict_files = sorted(glob('./다운로드/235801_2021 농산물 가격예측 AI 경진대회/public_data/dict/*.pkl'))

private_dict_list = []
for dict_file in dict_files:
    with open(dict_file, 'rb') as f:
        private_dict = pickle.load(f)
    private_dict_list.append(private_dict)
    
    
private = None

for private_dcit in private_dict_list:
    if private is None:
        private = pd.DataFrame(private_dcit)
    else:
        private = pd.concat([private, pd.DataFrame(private_dcit)])
        
private.reset_index(drop=True, inplace=True)

def get_number(text):
    return re.sub("[^0-9]", "", str(text))

private['date'] = private['date'].apply(lambda x : get_number(x))
private['date'] = private['date'].apply(lambda x : processing_date(x))

private['date'] = pd.to_datetime(private['date'])

data = private.sort_values(by=['date'])
data.reset_index(drop=True, inplace=True)
data.to_csv('real_data.csv', index=False)

/home/leeyongseong/다운로드/myanaconda/envs/test/lib/python3.7/multiprocessing/pool.py:44: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  return list(map(*args))


In [8]:
data.tail(50)

,date,배추_거래량(kg),배추_가격(원/kg),배추_COSTm,배추_COSTs,배추_SAN_NM,배추_WHSAL_NM,배추_CMP_NM,배추_KIND_NM,배추_QTYs,...,샤인마스캇_COSTs,샤인마스캇_SAN_NM,샤인마스캇_WHSAL_NM,샤인마스캇_CMP_NM,샤인마스캇_KIND_NM,샤인마스캇_QTYs,샤인마스캇_QTYm,샤인마스캇_DANQs,샤인마스캇_DANQm,샤인마스캇_LV_NM
1800,2021-09-14,2339571.90,1028.960184,15165.194434,38686411.0,강원 평창군,수원도매시장,수원청과,고냉지배추,242660.0,...,161303091.0,경북 경산시,수원도매시장,수원청과,샤인마스캇,215102.0,32.769957,16809.30,2.560832,.
1801,2021-09-15,2281838.40,972.565633,11370.148162,29698827.0,강원 평창군,부산반여도매,부산중앙청과,기타배추,237927.0,...,159827800.0,경북 김천시,부산반여도매,부산중앙청과,샤인마스캇,215056.0,33.290402,16818.10,2.603421,특
1802,2021-09-16,2220560.10,972.590469,11325.655603,29005004.0,강원 평창군,부산반여도매,부산중앙청과,기타배추,233567.0,...,155262502.0,경북 경산시,부산반여도매,부산중앙청과,샤인마스캇,204682.0,33.904588,15976.80,2.646480,특
1803,2021-09-17,2104179.00,928.394759,14142.034212,35963193.0,강원 철원군,창원팔용도매시장,창원청과,기타배추,221263.0,...,179825290.0,경남 창녕군,창원팔용도매시장,창원청과,샤인마스캇,222209.0,34.233400,17189.00,2.648128,특
1804,2021-09-18,1014318.20,1377.193210,13659.717583,20120764.0,강원도 정선군,대전오정도매,대전청과,고냉지배추,111033.0,...,149483332.0,경상북도 상주시,대전오정도매,대전청과,샤인마스캇,166906.0,34.420705,13005.70,2.682141,.
1805,2021-09-19,128020.60,1144.799196,10749.282946,2773315.0,울산 남구,울산도매시장,울산중앙청과,봄배추,17175.0,...,27212100.0,경남 거창군,창원팔용도매시장,창원청과,샤인마스캇,30785.0,37.959309,2214.00,2.729963,특
1806,2021-09-20,94185.00,1458.900037,11116.424581,1989840.0,강원도 강릉시,인천삼산도매,경인농산,기타배추,12118.0,...,66462900.0,경북 경산시,부산반여도매,동부청과,샤인마스캇,78677.0,36.357209,5893.50,2.723429,상
1807,2021-09-22,0.00,0.000000,0.000000,0.0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0.000000,0.00,0.000000,0
1808,2021-09-23,20216.00,1464.405125,17214.352941,1755864.0,경상북도 청송군,대구북부도매,농협북대구(공),쌈배추,2485.0,...,3152700.0,경북 경산시,원주도매시장,원주원협(공),샤인마스캇,3799.0,32.470085,286.00,2.444444,특
1809,2021-09-24,1972049.60,793.070473,13309.013852,24981019.0,전북 익산시,익산도매시장,이리청과,쌈배추,204155.0,...,124189500.0,경상북도 김천시,대구북부도매,대구중앙청과,샤인마스캇,199183.0,38.055598,13157.00,2.513756,특
